In [ ]:
from folktables.acs import adult_filter
from folktables import ACSDataSource, BasicProblem, generate_categories
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import balanced_accuracy_score, matthews_corrcoef
from sklearn.model_selection import KFold
import scipy.optimize as opt
import matplotlib.pyplot as plt

### 1. Load and Preprocess the data
We are going to work with the [Folktables](https://github.com/socialfoundations/folktables#quick-start-examples) dataset (*you have already worked with it*). I have chosen some variables for you, but you can add more (*if you like to*) - here is the [full list](https://www2.census.gov/programs-surveys/acs/tech_docs/pums/data_dict/PUMS_Data_Dictionary_2021.pdf) of variables (some of them do not exist in `ACSDataSource`). 

Today we are going to debias a regression model using the `SEX` variable. Your model should predict the *Total person's income*  (I've digitized  it in  `target_transform=lambda x: x > 25000`, you can choose another threshold).


* If you code is slow - you can subsample data (aka reduce the number of the samples)

In [2]:
data_source = ACSDataSource(survey_year='2018', horizon='1-Year', survey='person')
acs_data = data_source.get_data(states=["CA"], download=True)

ACSIncomeNew = BasicProblem(
    features=[
        'AGEP', # include AGE
        'COW', # include class of worker
        'SCHL', # include school education
        'WKHP', # include reported working hours
        'SEX', # include sex
        # some random, possibly noisy
        'PWGTP', # person weight
        'JWMNP', # travel time to work
    ],
    target='PINCP',
    target_transform=lambda x: x > 25000,    
    group='SEX',
    preprocess=adult_filter,
    postprocess=lambda x: np.nan_to_num(x, -1),
)

Here is a small snippet to get the names of the categorical variables - I convert categoricals into one-hot encoded (*you don't have to, depending on what assumptions you use about the data*). **Don't forget to normalise the continious features (if you plan to use Cross-Validation features should be normalized per fold, aka not in the global table).**

In [ ]:
definition_df = data_source.get_definitions(download=True)
categories = generate_categories(features=ACSIncomeNew.features, definition_df=definition_df)
# here I convert categoricals into one-hot encoded (you don't have to, depending on what assumptions you use about the data)
features, labels, groups = ACSIncomeNew.df_to_pandas(acs_data, categories=categories, dummies=True)
########### Normalize continious features
## YOUR CODE (if relevant)
###########
features.head()

Spliting data into train-test. **Again, if you plan to use Cross-Validation then you should normalise features only inside of a fold**.

In [4]:
X_train, X_test, y_train, y_test, group_train, group_test = train_test_split(
    features.values, labels.values.reshape(-1), groups.values.reshape(-1), test_size=0.3, random_state=0, shuffle=True)

N = 1000 ### I am subsampling because it is slow on my machine
X_train = X_train[:N]
y_train = y_train[:N]

### 2.  Regression model (without Fairness constraints)
Let's first train a simple **Logistic Regression**. 
1. Use L2 penalty to train the model (you should find the optimal value for the regularizer)
2. Calculate the total performance metric
3. Calculate and compare the perforformanc metric for each `SEX` group (use your favourite metric introduced during the course).

In [ ]:
##########
### YOUR CODE HERE
##########

### 3. Constrained Regression Model 
Now let's try to include the [Fairness Constraint](https://arxiv.org/abs/1706.02409)! You'll have to implement couple of things from scratch (as it is tricky to add a custom constraint function in `sklearn`.  To optimise the cost function let's use `scipy.optimize.fmin_tnc`. To calculate gradient you can use `fprime` attribute):
1. Logistic Regression
2. L2 penalisation
3. **Individual** Fairness Constrained

When you are finished with implementation - you should evaluate performance on multiple choices of fairness weight, $\lambda$.

#### Detailed breakdown
The INDIVIDUAL constraint constraint looks like this:

$$ 
f(\beta,S) = \left( \frac{1}{n_1 n_2} \sum\limits{(x_i,y_i)\in S_1 \\ (x_i,y_i)\in S_2} d(y_i,y_j) (\beta^T  \textbf{x}_i - \beta^T \textbf{x}_j)^2  \right) 
$$


For the constrained optimization we have to solve a problem on the form:

$$ \min_\beta \left( \ell (\beta,S) + \lambda f(\beta,S)  +\gamma \Vert \beta \Vert_2 \right) $$ 

where $\ell$ is some loss function, $f$ is the constraint function, and the $\gamma \Vert \beta \Vert_2 $ is L2 regularization (we use it to avoid overfitting).
(Basically we are minimizing the Lagrangian $\mathscr{L} = \ell (\beta,S) + \lambda f(\beta,S)  +\gamma \Vert \textbf{x} \Vert_2$ with respect to $\beta$ - in ML literature $\mathscr{L}$ is often denoted as J)

Because we are doing classification we are going to use logistic regression. The log loss function is:
$$
\ell = \frac{1}{m}\sum_i^m\left[ -y_i \log(g(x_i)) - (1-y_i)\log(1-g(x_i)) \right], \text{where } g(x_i) = \frac{1}{1+\exp(-\beta_i x_i)}
$$

For the distance function we follow the approach from Berk et al. (2017) and set:
$$d(y_i,y_j) = \begin{cases}
            1, &         \text{if } y_i=y_j,\\
            0, &         \text{if } y_i\neq y_j.
    \end{cases}$$
    
To minimize the total loss function we also need to estimate the gradient of $\mathscr{L}$ with respect to $\beta$. Here to update the $\beta$ values we are just going the gradient's without the fairness constraing - this will make our lives considerably easier. The j'th element of the gradiend is defined as follows:
$$
\frac{\partial \mathscr{L}}{\partial \beta_j} \approx \frac{1}{m}\left( \sum_i  (g(x_i) - y_i) x[j] \right)+ 2\gamma \beta_j
$$

##### A little clarification and tips:
1. In order to simplify the exercise - we cut some corners. *Ideally* we should calculate the gradient in respect to the *individual fairness*. The gradient takes into the account only logistic and l2 loss (aka, parameters are updated based on those). At the same time, our *cost* has a *individual fairness* included. When the update of the parameters stops decreasing the cost, the `fmin_tnc` is going to stop optimisation. So our implementation is not entirely correct.
2. In case you want to have a more correct implementation, you can do `opt.fmin_tnc(func=compute_cost, x0=betas, fprime = None, approx_grad= True, ...)`. It is quite long, but you can still do it
3. I also suggest setting `ftol=1e-5`. 
4. Don not apply l2-regularization on the intercept (when you calculate the gradient).
5. You should include $x_0 = 1$ in your data, for each observation (when it comes to the manual implementation of logistic regression) to include bias (i.e. weight $\beta_0$).
6. To keep the exercise simpler, let's fix $\gamma = 1e-5$.
7. Try $lambda$ is a range from around $1$ to $1e5$.

In [8]:
def sigmoid(x, eps = 1e-10):
    """
    This is logistic regression
    f = 1/(1+exp(-beta^T * x))
    This function assumes as input that you have already multiplied beta and X together
    """
    NotImplemented

def logistic_loss(y_true, y_pred, eps = 1e-10):
    """
    Loss for the logistic regression, y_preds are probabilities
    eps: epsilon for stability
    """
    NotImplemented

def l2_loss(beta):
    """
    L2-Regularisation
    """
    NotImplemented

def fair_loss(y, y_pred, groups):
    """
    Group fairness Loss
    """
    n = y.shape[0]
    n1 = np.sum(groups == 1)
    n2 = np.sum(groups == 2)
    cost = 0
    NotImplemented
    return (cost/(n1*n2))

def compute_gradient(beta,X,y, groups, _lambda,_gamma):
    """Calculate the gradient - used for finding the best beta values. 
       You do not need to use groups and lambda (fmin_tnc expects same input as in func, that's why they are included here)"""
    grad = np.zeros(beta.shape)
    NotImplemented

    for i in range(len(grad)):
        if i == 0: # we do not want to regularize the intercept
            grad[i] =  ...
        else:
            grad[i] = ...
        
    return grad

def compute_cost(beta ,X,y, groups, _lambda, _gamma):
    """Computes cost function with constraints"""
    NotImplemented
    probs = sigmoid(X.dot(beta))
    loss = log_loss + _lambda * fair_loss(y,X.dot(beta), groups) + _gamma * l2_loss(beta[1:])





beta = np.random.rand("number of features here")
lambdas = np.linspace(1,1e5, 20)

########## This is the optimisation function
#result = opt.fmin_tnc(func=compute_cost, x0=betas, fprime = compute_gradient, maxfun = 500,
#                          args = (X,y, groups, lambda_,1e-5), xtol=1e-7, ftol=1e-5)